# Kinetic modeling – sinusoidal temperature
In this notebook we will look at the solution of a kinetic problem where the temperature varies sinusoidally.

In [ ]:
import math
from collections import defaultdict
from chempy import ReactionSystem
from chempy.kinetics.ode import get_odesys
from chempy.kinetics.rates import SinTemp
%matplotlib inline

In [ ]:
rsys = ReactionSystem.from_string("""
2 HNO2 -> H2O + NO + NO2; EyringParam(dH=85e3, dS=10)
2 NO2 -> N2O4; EyringParam(dH=70e3, dS=20)
"""
)

In [ ]:
st = SinTemp(unique_keys='Tbase Tamp Tangvel Tphase'.split())

In [ ]:
odesys, extra = get_odesys(rsys, include_params=False, substitutions={'temperature': st})

In [ ]:
init_conc = defaultdict(lambda: 0, HNO2=1, H2O=55)
params = dict(
    Tbase=300,
    Tamp=10,
    Tangvel=2*math.pi/10,
    Tphase=-math.pi/2
)
duration = 60

In [ ]:
def integrate_and_plot(system):
    result = system.integrate(duration, init_conc, params, integrator='cvode', nsteps=2000)
    result.plot(names='NO HNO2 N2O4 NO2'.split(), title_info=2)
    print({k: v for k, v in sorted(result.info.items()) if not k.startswith('internal')})

In [ ]:
integrate_and_plot(odesys)

Since we are using [pyodesys](https://github.com/bjodah/pyodesys) we can reformulate our ODE-system as an autonomous one. This can help for stiff systems since time will then be explicitly represented in the Jacobian matrix.

In [ ]:
autsys = odesys.as_autonomous()
assert len(autsys.dep) == len(odesys.dep) + 1

In [ ]:
autsys.exprs

In [ ]:
integrate_and_plot(autsys)